In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Read csv file.
dataset = pd.read_csv('../input/source-based-news-classification/news_articles.csv')

# Remove misiing values.
dataset.drop(['title_without_stopwords','text_without_stopwords','hasImage'],axis=1,inplace = True)


dataset = dataset.dropna()
x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Encoding data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
ordinal = OrdinalEncoder()
label = LabelEncoder()
x = ordinal.fit_transform(x)
y = label.fit_transform(y)


'''
dataset['type'].value_counts().plot.pie(figsize = (8,8), startangle = 80)
plt.title('Types of articles', fontsize = 20)
plt.axis('off')
plt.show()
'''

# Splitting dataset into testing data and train data.

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 1)

# Feature Scaling.

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10,criterion = 'entropy', random_state = 0)
classifier.fit(x_train,y_train)
#classifier.predict(sc.transform(x_train))

# Predicting the Test set results
y_pred = classifier.predict(x_test)
np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session